### Welcome

This is the first section of the introductory GeoIPS tutorial, which includes running GeoIPS 
using the CLI, and creating your first plugin! 


Link to these slides 
https://github.com/NRLMMD-GEOIPS/presentations

### Tutorial Scope

This tutorial does not address running GeoIPS in near real-time. GeoIPS plugins 
are intended to be developed and tested on a specific dataset, and setting up the 
real-time processing infrastructure is a separate conversation. 

This tutorial focuses on: 

  - Installing GeoIPS 
  - Running GeoIPS 
  - Product development and testing 


### To get the most out of this tutorial… 

- [Aug 2024 - Intro GeoIPS Tutorial - Google Slides](https://docs.google.com/presentation/d/151tR2hycoM3WqC9-7s9laMcCTZ9StY2fVLTpA_FwYBs/edit#slide=id.g2e736e3f9ff_3_94) 
- Please keep these slides up to copy/paste from as needed throughout the 
tutorial 
- Please copy/paste rather than typing everything word for word! 
  - Trick - Triple click to select a full code block for copy/paste! 
- [NRLMMD-GEOIPS/template_basic_plugin workshop-2024-beginner-solutions](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2024-beginner-solutions) 
- If all else fails, you can access the solutions on github.com 

## System requirements

- **CPU:** 1 CPU
- **RAM:** 40GB as the notebook is written, but could use more if modified.
  Reading the entire full-disk ABI image can take up to 100GB.
- **Disk Space:** 3GB storage space.

***To see the default storage location for your system, run the cell below.***
To change the default location, change the value of `tmp_root` in the following cell.

In [ ]:
import tempfile
from pathlib import Path

# Set this to a different path if you would like to use a different location
# `tempfile.gettempdir()` returns a different location on different OS.

# tmp_root = "/use/this/path/instead"
tmp_root = Path(tempfile.gettempdir()) / "geoips_tutorial_tempdirs"

print(f"Notebook temporary storage path: {tmp_root}")

## Important notes
This notebook downloads approximately 6GB of data. If `geoips` or `geoips_clavrx` are not <br />
already installed, make sure to install them via `pip install --upgrade geoips geoips_clavrx`

This notebook makes an attempt at cleaning up after itself, but it is
recommended that, when done using this notebook, you check to be sure that the
directory reported by the next cell has been deleted.

## Setting up your Environment

**If you don't have an environment already set up, please follow the steps 2 and 3 of the instructions linked [here](https://nrlmmd-geoips.github.io/geoips/getting-started/installing/index.html) based on the architecture of your machine.**

You can quit following the instructions after you've activated your geoips environment via <br />
``conda activate geoips``.

Run the cell below to set up your environment. This will initialize a session-specific <br />
storage directory, add it to the global notebook environment, and add a hook that attempts <br />
to clean up the temporary storage when the notebook is closed.

❗
***Important:*** While this notebook makes an effort to clean up after itself, if you <br />
are running this notebook on your own system, ***it is advisable tomanually delete the <br />
temporary directory reported above when you are done using the notebook.***

In [ ]:
import dotenv
from IPython import get_ipython
import os

from utils import notebook_environment

# Sets up the environment and sets a global variable named `temp_dir`
notebook_environment.setup(tmp_root)

with open("./.env", "w") as env_file:
    env_file.writelines(
        [
            f"GEOIPS_TESTDATA_DIR={get_ipython().user_ns['temp_dir']}/test_data\n",
            f"GEOIPS_OUTDIRS={get_ipython().user_ns['temp_dir']}/outdirs\n",
            f"GEOIPS_PACKAGES_DIR={get_ipython().user_ns['temp_dir']}\n",
            f"GEOIPS_REBUILD_REGISTRIES=True",
            f"MY_PKG_DIR={get_ipython().user_ns['temp_dir']}/cool_plugins\n",
            "MY_PKG_NAME=cool_plugins"
        ],
    )

dotenv.load_dotenv("./.env", override=True)

if not os.path.exists(os.environ["GEOIPS_TESTDATA_DIR"]):
    os.makedirs(os.environ["GEOIPS_TESTDATA_DIR"])
if not os.path.exists(os.environ["GEOIPS_OUTDIRS"]):
    os.makedirs(os.environ["GEOIPS_OUTDIRS"])


### Installing Appropriate Test Datasets

For this tutorial, you'll need GeoIPS test datasets that can be used to produce imagery 
or scientific datasets. 

If just attending the beginner tutorial, you'll need:

  - `test_data_clavrx`

These datasets might take a while to download so go ahead and download the appropriate 
datasets based on the following commands,.

In [ ]:
%%bash

# This is needed for both the beginner and advanced tutorial
geoips config install test_data_clavrx

### Introduction to GeoIPS

GeoIPS is a plugin-based system for processing geolocated data: 
  - produce imagery in several formats (most often PNG). 
  - produce output data products in NetCDF4 format. 
  - extended to add other output formats via plugins. 

![1111𝝁 infrared imagery](./images/conus_infrared.png)

![Himawari-9 CLAVR-x Cloud-Top-Height](./images/ahi_cloud_top_height.png)

![ABI CLAVR-x Cloud Top Height](./images/abi_cloud_top_height.png)

GeoIPS is almost entirely composed of plugins:

  - GeoIPS can be extended by developing new plugins in external python packages. 
  - No need to edit the main GeoIPS code to add new functionality. 
  - Most types of functionality in GeoIPS can be extended (and if something can’t be 
  extended, and you think it should, let us know!). 


### Vocabulary

**YAML**
  - "Yet Another Markdown Language" 
  - A human-readable data serialization language that is often used for writing configuration files 

**Plugin**
  - A Python module or YAML file that defines GeoIPS functionality 
  - Stored in an installable Python package that registers its plugin payload with GeoIPS 

**Interface**
  - A class of Python plugins that modify the same type of functionality within GeoIPS 
  (e.g., “the algorithms interface” or “the colormappers interface”) 
  
**Family**
  - A subset of an interface whose plugins accept different sets of arguments/properties 
  - Planning to deprecate this 'artifact' in the near future.


### Commonly used GeoIPS Plugin Types 

**algorithm**
  - Implements a function that modifies data and outputs new data

**colormapper**
  - Defines a method of applying a colormap to imagery

**feature_annotator**
  - Defines how to plot map features (e.g. coastlines, borders, rivers, etc.)

**gridline_annotator**
  - Defines how to plot gridlines and labels

**interpolator**
  - Defines a method of interpolating data to a sector

**output_formatter**
  - Defines a method for plotting imagery or outputting a data file 

**procflow**
  - Defines the order of operations to use when producing a product from GeoIPS 

**product**
  - Defines how to produce a specific product as a combination of other plugins
  - Uses other plugins (e.g. algorithm, colormapper, interpolator)

**product_defaults**
  - Defines a default set of plugins and arguments for producing a specific type of product 
  - Can be reused across multiple products to allow consistency between similar products 

**reader**
  - Defines a data reader 

**sector**
  - Describes an domain for reprojection of data 

## Hands On: Modify a plugin template to create your own installable plugin package  


### Get the Template Repo

Run the following series of commands to get the template repository.

In [ ]:
%%bash

cd $GEOIPS_PACKAGES_DIR
git clone --no-tags --single-branch $GEOIPS_REPO_URL/template_basic_plugin.git
# Rename your package
mv template_basic_plugin/ $MY_PKG_NAME
cd $MY_PKG_NAME
# No longer point to github.com template_basic_plugin.git
git remote remove origin
git commit -m "renamed my_package to cool_plugins"

### Update the Package Name

Ensure you’re in your package directory and look around.

This repository is set up with a working installable plugin called "my_package", so we 
just need to swap out the name and build upon what's already there! 

![Package listing](./images/cool_plugins_top_level.png)

This repository is set up with a working installable plugin called "my_package", so we 
just need to swap out the name and build upon what's already there!

### Structure of a Plugin Package

Rename the default plugin package directory to your new package name

In [ ]:
%%bash

cd $MY_PKG_DIR
git mv my_package $MY_PKG_NAME
tree -L 2

### Your directory structure should now look like this:

![cool_plugins directory structure](./images/cool_plugins_directory_structure.png)

In [ ]:
%%bash

cd $MY_PKG_DIR/cool_plugins
tree

![plugins directory structure](./images/plugins_directory_structure.png)

### Update Pertinent Files

1. Update README.md (`vim README.md`)
  - Find/replace all occurrences of @package@  with your package name 
  - Vim Tip :%s/@package@/cool_plugins/g 
  
**Note**: The @ symbols are for ease of searching, take them out when you put your 
package name in!

2. Update pyproject.toml (`vim pyproject.toml`, more on this in soon.)
  - Find/replace all occurrences of my_package  with your package name 

3. Add and commit your changes.

#### Note

Due to the nature of Jupyter Notebooks, we are not able to edit files in place (that is <br />
opening them up and manually editing files (such as YAML or TOML)). Instead we'll <br />
update these files with Python. Keep in mind if you were running this notebook locally <br />
via an IDE, you'd be able to edit these files manually rather than using Python.

In [ ]:
"""Overwrite cool_plugins' pyproject.toml and README.md with correct contents."""

import os

with open("./updated_files/pyproject.toml", "r") as rf:
    toml_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/pyproject.toml", "w") as wf:
    wf.writelines(toml_lines)

with open("./updated_files/README.md", "r") as rf:
    md_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/README.md", "w") as wf:
    wf.writelines(md_lines)

In [ ]:
%%bash

cd $MY_PKG_DIR
git add README.md pyproject.toml
git commit -m "Updated name of template plugin package to mine"

4. Install your package (-e means `editable` so we can edit the package after it is 
installed and changes will be reflected in the installed package)

Maybe mention a symlink -- similar to if you've symlinked your package to python installation path (site-packages)

In [ ]:
%%bash

pip install -e $MY_PKG_DIR

5. Register your plugins so GeoIPS knows where to find them

In [ ]:
%%bash

geoips config create-registries

### See what you just installed 

List all installed packages:

In [ ]:
%%bash

geoips list packages

List all installed plugins:

In [ ]:
%%bash

geoips list plugins

### A bit about pyproject.toml

Installing Python packages requires metadata that describes the package and how to 
install it. 

`pyproject.toml` defines this information for pip, including: 
  - Package name, version, description, license, etc. 
  - Which files should be contained in the package when installed 
  - How to build the package 

We make GeoIPS aware of our package using the `geoips.plugin_packages` namespace.
This allows GeoIPS to find all plugins within packages registered to this namespace.

GeoIPS automatically identifies all plugins defined within a plugin package via a plugin
registry. You can manually create these files via `geoips config create-registries`, 
however, GeoIPS will automatically create these files if a requested plugin cannot be
found. This usually occurs the first time GeoIPS is initialized.

**NOTE** for plugin registries to write successfully: 

1. All installed  plugin names within a given interface must be unique 

2. All installed plugins must be formatted and defined correctly 

We will make use of this more later! Modify plugin template solutions on 
[NRLMMD github.com](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2023-solutions).

```
[tool.poetry.plugins."geoips.plugin_packages"]
"cool_plugins" = "cool_plugins" 
``` 

## GeoIPS Command Line Interface (CLI) Tutorial

Please follow this [Jupyter Notebook](./CLI_Tutorial.ipynb) for instructions on how to 
make use of the GeoIPS CLI.

## Hands on: Create a Product Plugin (products  YAML-based interface) 

Solutions on 
[NRLMMD github.com](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2023-solutions).

### First Product Plugin: Cloud Top Height (CTH) from CLAVR-x data

- Copy the existing product plugin to a new file to modify 

In [ ]:
%%bash

cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/products
cp amsr2_using_product_defaults.yaml my_clavrx_products.yaml

- Edit my_clavrx_products.yaml properties (I.e. `vim my_clavrx_products.yaml`)

    a. (Feel free to remove all lines preceded by `# @`) 
  
![Product top level keys](./images/product_top_level_keys.png)

All YAML plugins will begin with these same four properties!

In a future code cell we'll replace the YAML shown on the left of the image with the <br />
following in my_clavrx_products.yaml

```yaml
interface: products
family: list
name: my_clavrx_products
docstring: |
  CLAVR-x imagery products
```

### First Product Plugin: Cloud Top Height (CTH) from CLAVR-x data 

Edit the product specifications as shown below:

![Update product spec](./images/update_product_spec.png)

In the next code cell we'll replace the YAML shown on the upper left of the with the <br />
following in my_clavrx_products.yaml

```yaml
spec:
  products:
    - name: My-Cloud-Top-Height
      source_names: [clavrx]
      docstring: |
        CLAVR-x Cloud Top Height
      product_defaults: Cloud-Height
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"]
```

In [ ]:
"""Update the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]

updated_products["spec"]["products"] = [my_cloud_top_height]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)


In [ ]:
%%bash

cd $MY_PKG_DIR/cool_plugins/plugins/yaml/products
pwd
cat my_clavrx_products.yaml

### Command: `geoips describe`

 
Let's use the CLI to get more information about the plugin we just created. 

You'll notice that the plugin registry is rebuilt, as GeoIPS was unable to locate the 
plugin we just created (since it was not already in the registry).

In [ ]:
%%bash

geoips config create-registries
geoips describe product clavrx.My-Cloud-Top-Height

If you need a reminder what your new product is called, you can always check via:

In [ ]:
%%bash

geoips ls products -p cool_plugins

For any description of a product plugin, this is the generic format to follow via the CLI:

`geoips describe <interface_name> <source_name>.<plugin_name>`

### `family: null`

You may have noticed that your product’s family was `null` in the output of 

`geoips describe product clavrx.My-Cloud-Top-Height`. 

This isn’t the actual case. 

Products are the only plugin that can depend on another type of plugin 
(product_defaults), and that is where this information lies. 

For now, an easy way to check the family of your product is by describing its product 
default plugin. 

The family of the derived product_default is the same family as the product plugin.

In [ ]:
%%bash

geoips desc pdef Cloud-Height

### Use your new product!

To use the plugin you just created, we'll make use of the `geoips run` command.

- GeoIPS is called via a command line interface (CLI), as we've shown previously.
- The main command that you will use is `geoips run single_source`, which will run your 
  data through the specified procflow using the specified plugins 
- It's easiest to do this via a script, and scripts are stored in your plugin package's 
  `tests/` directory because they can be used later to regression test your package 
- Since we're running this in a notebook, we can easily run these commands with a code cell!

In [ ]:
%%bash

geoips run single_source \
    $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Top-Height \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --sector_list conus

### Viewing the log output

This will write some log output.  If your script succeeded it will end with 
`INTERACTIVE: Return Value 0`.

To view your output, look for a line that says `SINGLESOURCESUCCESS`.

Open the PNG file that this script produces.

If successful, the output image should look like this:

![CLAVR-x CONUS My-Cloud-Top-Height](./images/clavrx-conus-my-cloud-top-height.png)

### A word about Product Defaults 

- GeoIPS has a number of product_defaults plugins defined to help you not reinvent the 
  wheel, **but**
    - You can override any of the product defaults within your product definition 
    - You can absolutely define all of the available options within your product plugin 
- [Pre-defined CLAVR-x product defaults](https://github.com/NRLMMD-GEOIPS/geoips_clavrx/tree/main/geoips_clavrx/plugins/yaml/product_defaults) (part of the CLAVR-x plugin) 
- [Pre-defined GeoIPS product defaults](https://github.com/NRLMMD-GEOIPS/geoips/tree/main/geoips/plugins/yaml/product_defaults) 
- If you have product definition parameters that you want to reuse (i.e. if you're 
  copy/pasting product definition parameters!),consider creating a product default for 
  your plugin 

  ![Cloud-Height Product Defaults](./images/Cloud-Height-product_defaults.png)

### Different Implementations of Product Defaults within a Product 

In your product you can use the product_defaults verbatim. 

![Product using product default](./images/product_using_product_default.png)

### Different Implementations of Product Defaults within a Product 

You can also override just some parts of the product_defaults. 

In this example, we override the `algorithm` plugin contained in the Cloud-Height 
product_defaults, with our own specification. 

![Product default overridden](./images/product_default_overridden.png)

### Different Implementations of Product Defaults within a Product

We also have the option to define a product without using product_defaults.

To do this: 
  - remove the `product_defaults` property 
  - add the `family` property

![Product fully specified](./images/product_fully_specified.png)

## Hands on: Add Additional Products to Your Product Plugin 


### Add Cloud Base Height (CBH) to the CLAVR-x Product Definition

- Using your definition of `My-Cloud-Top-Height` as an example, create a product 
  definition for `My-Cloud-Base-Height`

**Helpful Hints:** 
- The relevant variable in the CLAVR-x output file (and the equivalent GeoIPS reader) 
  is called `cld_height_base`
- The Cloud-Height product_default  can be used to simplify this product 
  definition (or you can DIY or override if you'd like!) 

In [ ]:
"""Add My-Cloud-Base-Height to the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]
my_cloud_base_height = updated_products["spec"]["products"][1]

updated_products["spec"]["products"] = [my_cloud_top_height, my_cloud_base_height]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)

### First Product Plugins: CTH and CBH from CLAVR-x data (my_clavrx_products.yaml) 

After running the code cell above, the `spec` portion of `my_clavrx_products.yaml` <br />
should now look like this:

```yaml
spec:
  products: 
    - name: My-Cloud-Top-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"] 
```

In [ ]:
%%bash

# verify that the contents of my_clavrx_products.yaml have the spec shown above
cd $MY_PKG_DIR/cool_plugins/plugins/yaml/products
cat my_clavrx_products.yaml

### Add Cloud Depth to the CLAVR-x Product Definition 

- Using your definitions of `My-Cloud-Top-Height` and `My-Cloud-Base-Height` as examples, 
  create a product definition for `My-Cloud-Depth`

**Helpful Hints:** 
- We will define Cloud Depth for this tutorial as the difference between CTH and CBH 

**Note:**
- This is meant to challenge you a bit! Give it a try and we'll go over the solution in 
  the next markdown block 

### Cloud Depth Product with Default Algorithm Applied (my_clavrx_products.yaml) 

After running the code block below, the `spec` portion of `my_clavrx_products.yaml` <br />
should now look like this:

```yaml
spec:
  products: 
    - name: My-Cloud-Top-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"]
    - name: My-Cloud-Depth
      source_names: [clavrx]
      docstring: |
        CLAVR-x Cloud my Cloud Depth
      product_defaults: Cloud-Height
      spec:
        variables: ["cld_height_acha", "cld_height_base", "latitude", "longitude"]
```

We now have two variables, but if we examine the Cloud-Height Product Defaults we see 
that it uses the `single_channel` algorithm.

This algorithm just manipulates a single data variable and plots it. 

Therefore, we need a new algorithm plugin!

In [ ]:
"""Add My-Cloud-Depth to the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]
my_cloud_base_height = updated_products["spec"]["products"][1]
my_cloud_depth = updated_products["spec"]["products"][2]

updated_products["spec"]["products"] = [
    my_cloud_top_height,
    my_cloud_base_height,
    my_cloud_depth,
]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)

In [ ]:
%%bash

# verify that the contents of my_clavrx_products.yaml have the spec shown above
cd $MY_PKG_DIR/cool_plugins/plugins/yaml/products
cat my_clavrx_products.yaml

### Validate your new product plugins

Before you go ahead creating your new algorithm plugin, let's validate that the product
plugins we just created are working as expected.

In [ ]:
%%bash

geoips config create-registries
geoips describe product clavrx.My-Cloud-Top-Height
geoips describe product clavrx.My-Cloud-Base-Height
geoips describe product clavrx.My-Cloud-Depth

### Command: `geoips list`

Additionally, before creating a new algorithm, let’s make sure your products are 
registered within GeoIPS. 

In [ ]:
%%bash

geoips list products -p cool_plugins

## Hands on: Create an Algorithm Plugin 

1. Copy the existing algorithm plugin to a new file to modify

In [ ]:
%%bash

cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/algorithms
cp pmw_89test.py my_cloud_depth.py

![Updating top level portions of new algorithm](./images/updating_algorithm_top_level.png)

Shown in the image above, let's update our `my_cloud_depth` algorithm with the following:

```python
"""Cloud depth product.

Difference of cloud top height and cloud base height.
"""

import logging

from xarray import DataArray

LOG = logging.getLogger(__name__)

interface = "algorithms"
family = "xarray_to_xarray"
name = "my_cloud_depth"
```

### Updating your Algorithm

Algorithms (alongside all `module-based` plugins) must include a `call()` function.

This function is what is called when the algorithm is used.

The `cal()` function's signature is determined by the algorithm's family.

To create your new algorithm, add the `"scale_factor"` parameter to the call signature.

Replace the signature of the `call()` function and its docstring with the following. You 
can remove the comments if desired.

```python
def call(
    xobj,  # Xarray Dataset holding DataArrays
    variables, # list of required input variables for the algorithm. These are ordered as specified in your product plugin.
    product_name,
    output_data_range,
    scale_factor,  # Adding a scale factor here for use in converting input meters to output kilometers
    min_outbounds="crop",
    max_outbounds="mask",
    norm=False,
    inverse=False,
):
    """My cloud depth product algorithm manipulation steps."""
```

This is where the actual data manipulation occurs. Make sure to index the variable list 
to the order of the variables you defined in your product.

Replace the contents of the `call()` function with the following. You can remove the 
comments if desired.

```python
    # Variables in the order defined in your Cloud-Depth Plugin
    cth = xobj[variables[0]]
    cbh = xobj[variables[1]]

    out = (cth - cbh) * scale_factor

    from geoips.data_manipulations.corrections import apply_data_range

    # Data manipulation: Anything you want!
    data = apply_data_range(
        out,
        min_val=output_data_range[0],
        max_val=output_data_range[1],
        min_outbounds=min_outbounds,
        max_outbounds=max_outbounds,
        norm=norm,
        inverse=inverse,
    )
    xobj[product_name] = DataArray(data)

    return xobj
```

Once complete, let's describe the plugin you just created to make sure it registered
appropriately.

In [ ]:
"""Replace the contents of my_cloud_depth.py with the correct depth algorithm."""

import os

with open("./updated_files/my_cloud_depth.py", "r") as rf:
    python_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/modules/algorithms/my_cloud_depth.py", "w") as wf:
    wf.writelines(python_lines)

In [ ]:
%%bash

geoips config create-registries
# In case you forgot the name of your plugin, you can run:
geoips ls algs -p cool_plugins
# Now let's describe that plugin
geoips describe algorithm my_cloud_depth

### Override Product Defaults to Use Our Algorithm (my_clavrx_products.yaml) 

Let's revisit our My-Cloud-Depth product definition to use the algorithm we just created.

In [ ]:
%%bash

cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/products
vim my_clavrx_products.yaml # or another method of editing a file (if you're using an IDE, you can just open the file and edit it)

### Overriding the My-Cloud-Depth Algorithm

As mentioned earlier, we can override a product's defaults by adding new information to <br />
that product's `spec` object. 

This was complete when we previously added our `My-Cloud-Depth` product, however it's <br />
important to understand how this functionality works. Since that product inherits from <br />
the `Cloud-Height` product_defaults plugin, `My-Cloud-Depth` assumes it would use the <br />
`single_channel` algorithm. We want to override that with the algorithm we just created. <br />

You can do that by implementing the code below.

```yaml
spec:
  products:
    - name: My-Cloud-Top-Height
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"]
    # Update the code block below
    - name: My-Cloud-Depth 
      source_names:  [clavrx] 
      docstring:  |
        CLAVR-x Cloud my Cloud Depth 
      product_defaults:  Cloud-Height 
      spec:
        variables:  ["cld_height_acha", "cld_height_base", "latitude", "longitude"] 
        # Algorithm override portion
        algorithm: 
          plugin:
            name: my_cloud_depth # The name we assigned our algorithm when we defined it
            arguments: 
              output_data_range:  [0, 20]
              scale_factor:  0.001
```

### Hands on: Using Your Algorithm Plugin

Run the following command to make use of your new algorithm plugin, referenced in our <br />
`My-Cloud-Depth` product.

In [ ]:
%%bash

geoips run single_source \
  $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Depth \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --sector_list conus

### Using Your Algorithm Plugin 

Run your script.

This will output a bunch of log output.  

If your script succeeded it will end with `INTERACTIVE: Return Value 0`.

To view your output, look for a line that says `SINGLESOURCESUCCESS`.

Open the PNG file. It should look like this:

![My Cloud Depth Image](./images/my-cloud-depth.png)
